<a href="https://colab.research.google.com/github/Raikils/term_paper/blob/main/ISR_ResNet50_GAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import random
import pandas as pd
import csv
import os
import random

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
!wget http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar

In [ ]:
!tar -xvf "indoorCVPR_09.tar"

In [ ]:
files = os.listdir('Images')
data = []
for label in files:
  data += [label + '/' + x for x in os.listdir(os.path.join('Images', label))]
random.seed()
data = random.sample(data, len(data))
train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]
valid_data = train_data[int(len(train_data)*0.8):]
train_data = train_data[:int(len(train_data)*0.8)]

In [ ]:
with open('train.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in train_data:
    class_name=item[:item.index('/')]
    img_name='Images/'+item
    csvw.writerow([img_name,class_name])

with open('test.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in test_data:
    class_name=item[:item.index('/')]
    img_name='Images/'+item
    csvw.writerow([img_name,class_name])

with open('valid.csv','w',newline='') as f:
  csvw=csv.writer(f)
  csvw.writerow(['filename','class'])
  for item in valid_data:
    class_name=item[:item.index('/')]
    img_name='Images/'+item
    csvw.writerow([img_name,class_name])

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,vertical_flip=True
                                                             ,zoom_range=0.2,rotation_range=360
                                                             ,width_shift_range=0.1,height_shift_range=0.1
                                                             ,channel_shift_range=50
                                                             ,brightness_range=(0,1.2)
                                                             ,preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)
test_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=keras.applications.imagenet_utils.preprocess_input)

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
valid_df = pd.read_csv("valid.csv")

In [ ]:
batch_size=50
train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_df,
      directory='.',
      x_col="filename",
      y_col="class",
      target_size=(224, 224),
      batch_size=batch_size,
      class_mode='categorical',shuffle=True)
test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='.',
        x_col="filename",
        y_col="class",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)

valid_generator = valid_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='.',
        x_col="filename",
        y_col="class",
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',shuffle=True)
train_img_num=len(train_generator.filenames)

In [ ]:
keras.backend.clear_session()
shape=(224,224,3) 
input_tensor=keras.Input(shape=shape)
base_model=keras.applications.ResNet50(input_tensor=input_tensor,weights="imagenet",include_top=False)
avg=keras.layers.GlobalAveragePooling2D()(base_model.output)
preds=keras.layers.Dense(67,activation='softmax',
                          kernel_initializer=keras.initializers.RandomNormal(mean=0.0,stddev=0.01),
                          bias_initializer=keras.initializers.Zeros(),)(avg)
model=keras.Model(inputs=base_model.input, outputs=preds)  
for layer in model.layers:
  layer.trainable = True
model.summary()

In [ ]:
name = 'ResNet50_GAP'

In [ ]:
!mkdir "models"
filepath="models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
filepath="gdrive/MyDrive/data/models/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%name
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
optimizer=keras.optimizers.SGD(momentum=0.9,learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator, epochs=80,shuffle=True,callbacks=callbacks_list, validation_data=valid_generator)

In [ ]:
model.evaluate_generator(test_generator)